In [3]:
%reload_ext autoreload
%autoreload 2

from book_functions import *
from signin_config import *

from tqdm import tqdm
import pandas as pd
import re
import requests
import signin_config
import pickle
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from imblearn.over_sampling import SMOTE
import time
from selenium.webdriver.chrome.options import Options

Using TensorFlow backend.


**Psuedo-code process:**

top_200_lists = [ agg site urls ]

full_books_list = [ book dictionaries ]

from top_200 :
	200_book_dic = scrape_intial_info(top_200)
	full_books_list += 200_book_dic


for each_dict in full_books_list:
	each_dict += get_new_info( )

	every 50 books:
		latest = full_books_list[ last50 ]
		clean_and_bag_words( latest )
		save_those_50( latest )
		concat_to_saved( latest )
		rewrite_the_saved(saved_list)
 


# Get top 200 list urls from 1909 - 2019

In [79]:
top_200_lists = get_urls()
len(top_200_lists) # should be 111

111

# Inital scrape of books using top 200 lists - returns title, author, rating, num_ratings and id

In [120]:
#add webdriver and initial scrape function
driver = webdriver.Chrome()
initial_gr_signin(driver)

try:
    full_books_list
except:
    full_books_list = []

for top_200_list in top_200_lists[:5]:
    driver.get(top_200_list)
    dic_of_200_books = goodreads_list_scrape(driver)
    full_books_list += dic_of_200_books

In [119]:
first_books_df = pd.DataFrame(full_books_list)
first_books_df.to_csv('books_first_scrape.csv',index=False)

In [ ]:
# run cell only when restarting whole process - shouldn't be run locally anymore
# # pd.DataFrame([]).to_csv('updating_df.csv') 
# # pd.DataFrame([]).to_json('updating_df.json', orient='records') 

# Secondary scrape for img, description, genre, format, pages and isbn

**Also includes continual saving, with space for automated git pushing**

In [106]:
# run these first lines only if above has not already been run - saves time and energy
# driver = webdriver.Chrome()
# initial_gr_signin(driver)

for each_dict in full_books_list:
    idx = full_books_list.index(each_dict)
    
    each_dict.update(secondary_scrape(each_dict['id'],driver))
    each_dict.update(clean_and_bag_words(each_dict))
    
    if idx % 50 == 0 and idx != 0:
        latest = pd.DataFrame(full_books_list[idx-50:idx])
        save_those_50(latest, 'last_50')
        update_saved(add_to_saved_df(latest, 'updating_df.csv'), 'updating_df')
        push_to_git('last_50', 'updating_df',f'added 50 rows to df at idx {idx}')
        

# saves to full_books_list when finished
pd.DataFrame(full_books_list).to_csv('full_books_list.csv', index=False)
pd.DataFrame(full_books_list).to_json('full_books_list.json', orient='records')

FileNotFoundError: [Errno 2] File b'updating_df' does not exist: b'updating_df'

In [5]:
df = pd.read_csv('last_50.csv')

In [6]:
james_data = df[['id','authors', 'titles', 'description', 'img', 'genre']]

In [7]:
james_data

,id,authors,titles,description,img,genre
0,77390,L.M. Montgomery,"Anne of Avonlea (Anne of Green Gables, #2)","At sixteen, Anne is grown up...almost. Her gra...",https://i.gr-assets.com/images/S/compressed.ph...,"Classics, Fiction, Young Adult, Historical, Hi..."
1,5246,Edith Wharton,Ethan Frome,'He seemed a part of the mute melancholy lands...,https://i.gr-assets.com/images/S/compressed.ph...,"Classics, Fiction, Literature, Academic, School"
2,93101,Anonymous,The Arabian Nights,The tales of told by Shahrazad over a thousand...,https://i.gr-assets.com/images/S/compressed.ph...,"Classics, Fiction, Fantasy, Short Stories"
3,929782,Jack London,Martin Eden,Alternate cover edition of ISBN 9780140187724 ...,https://i.gr-assets.com/images/S/compressed.ph...,"Classics, Fiction, Literature, American"
4,17567,Gene Stratton-Porter,"A Girl of the Limberlost (Limberlost, #2)","Set amid Indiana's vast Limberlost Swamp, this...",https://i.gr-assets.com/images/S/compressed.ph...,"Classics, Fiction, Young Adult, Historical, Hi..."
5,259020,Henry James,The Golden Bowl,"'A thing to marvel at, a thing to be grateful ...",https://i.gr-assets.com/images/S/compressed.ph...,"Classics, Fiction, Literature, Literature, Ame..."
6,4711854,E.M. Forster,The Machine Stops,The Machine Stops is a science fiction short s...,https://i.gr-assets.com/images/S/compressed.ph...,"Science Fiction, Short Stories, Fiction, Class..."
7,179639,L. Frank Baum,"The Road to Oz (Oz, #5)","Meet Dorothy's new friends, the Shaggy Man, Bu...",https://i.gr-assets.com/images/S/compressed.ph...,"Fantasy, Classics, Childrens, Fiction"
8,38673,J.M. Barrie,Peter Pan in Kensington Gardens / Peter and Wendy,"In Peter Pan in Kensington Gardens, J.M. Barri...",https://i.gr-assets.com/images/S/compressed.ph...,"Classics, Fantasy, Fiction, Childrens"
9,19337,Beatrix Potter,The Tale of the Flopsy Bunnies,When the cupboard is bare at the Flopsy Bunny'...,https://i.gr-assets.com/images/S/compressed.ph...,"Childrens, Childrens, Picture Books, Classics,..."
